In [61]:
from tqdm import tqdm, trange
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import scipy as sp
import time
import sys
import importlib
import os
import hashlib
import subprocess
import gc
from datetime import datetime

from IPython import display
from IPython.display import clear_output
import copy
from copy import deepcopy
# from sklearn.metrics import roc_auc_score, roc_curve

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset

# from captum.attr import IntegratedGradients
# from captum.attr import LayerConductance
# from captum.attr import NeuronConductance

try:
    import cPickle as pickle
except:
    import pickle
    
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

experiment_folder="."
all_ligs_db_file=f"{experiment_folder}/all_ligands.pickle"

datafolder=f"{experiment_folder}/cached_reprs"

# sys.path.append(experiment_folder)

Bohr2Ang=0.529177249
RT=0.001985875*300 #kcal/mol


/tmp/ipykernel_613/1548949676.py:37: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# (Re)load Model and Dataset classes

In [54]:
if 'NNs' in sys.modules:
    importlib.reload(sys.modules['NNs'])
else:
    import NNs
from NNs import *

if 'computeDescriptors' in sys.modules:
    importlib.reload(sys.modules['computeDescriptors'])
else:
    import computeDescriptors
from computeDescriptors import *

# Load ligands and init Dataset

In [3]:
with open(all_ligs_db_file, 'rb') as f:
    ligs = pickle.load(f)
    
DB=CustomMolModularDataset(ligs)
print(f"{len(DB)} molecules with {DB[0][0].shape[0]} descriptors")

642 molecules with 6679 descriptors


## Remove constant and highly correlated features

In [46]:
allX=np.array([entry[0] for entry in DB])
print("Shape of all data:", allX.shape)

# exclude features that are always constant
keep_feature_indeces = np.where(np.logical_not(np.all(allX == allX[0,:], axis=0)))[0]
print(f"Variable (non-constant) features: {len(keep_feature_indeces)}")


# correlation
cormat=np.corrcoef(allX[:,keep_feature_indeces].transpose())
cormat -= np.tril(cormat) # remove self correlation and lower triangular
cormat=np.abs(cormat) # absolute values

# find any columns with high correlations
cor_threshhold = 0.8
high_cor_pairs = np.where(cormat>=cor_threshhold)
to_remove = np.unique(high_cor_pairs[0]) # indeces to the keep_feature_indeces array
print(f"Highly correlated (>={cor_threshhold}) features to remove: {len(to_remove)}")

# WARNING/TODO: this removes features too agressively.
# If feature B is corelated to A and C to B, but not C to A
# this will remove both B and C.
# Ideally if we remove B, we should retain C, but that requires more recursion

# remove the correlated features
keep_mask = np.ones(len(keep_feature_indeces), np.bool_)
keep_mask[to_remove] = 0
keep_feature_indeces = keep_feature_indeces[keep_mask]

print(f"Keapt features: {len(keep_feature_indeces)}")

Shape of all data: (642, 6679)
Variable (non-constant) features: 5180
Highly correlated (>=0.8) features to remove: 1739
Keapt features: 3441


## Reinitialize dataset object without the removed features

In [56]:
DB=CustomMolModularDataset(ligs, X_filter=keep_feature_indeces)
print(f"{len(DB)} molecules with {DB[0][0].shape[0]} descriptors remaining")

642 molecules with 3441 descriptors remaining


## Prep Training

In [57]:
normalize_x=True
shuffle_seed=12345678

#n_Epochs=2000
#hl_w=300
#hl_depth=2

n_Epochs=200
hl_w=20
hl_depth=1

init_learning_rate=5e-3
learning_rate_decay=10000 #order of magnitude in this many epochs
weight_decay=1e-3

normalize_x=True
X_filter=None
impfilt=None

weighted=True
use_dropout=True
shiftY=True

redo=False

batchsize=100
eval_batchsize=200
activation="relu"

eval_every=min(10, n_Epochs)
plot_every=min(1000, n_Epochs)
backup_models_every=1000

normalize_x=True
predict_all_ligs=True

save_folder="models/Perceptron"


# Train

In [62]:
allYdata=(np.array([float(lig.GetProp('dG')) for lig in ligs]))
minY=np.min(allYdata)
maxY=np.max(allYdata)

    #shuffle ligand indeces
all_idxs_shuffled=np.arange(len(ligs))
np.random.seed(shuffle_seed)
np.random.shuffle(all_idxs_shuffled) # in place transform


if(X_filter is not None):
    if(not os.path.exists(X_filter)):
        raise(Exception(f"No such file: {X_filter}"))
        
if(weighted):
    kde = sp.stats.gaussian_kde(allYdata)
    temp_Y=np.linspace(np.min(allYdata), np.max(allYdata), 20, endpoint=True)
    temp_kde=kde(temp_Y)
    C=1/np.mean(1/temp_kde)
    def get_weights(y):
        return(C/kde(y))
    weight_func=get_weights

    ###################debug###################
    #weights=weight_func(temp_Y)
    #print(temp_Y)
    #print(weights)
    #plt.plot(temp_Y, weights)
    #plt.show()
    #raise()
    ###################debug###################
else:
    weight_func=None

if(use_dropout):
    p_dropout=np.repeat(0.5, hl_depth+1)
else:
    p_dropout=np.repeat(0.0, hl_depth+1)

noise=0.0

train_clr="blue"
Xval_clr="purple"
unkn_clr="green"
summary_train_clr="darkred"
summary_unkn_clr="peru"


#generator for all data
full_dataset = DB

if(normalize_x):
    full_dataset.find_normalization_factors()
    print("Found normalization factors across all ligands", datetime.now(), flush=True)
if(predict_all_ligs):
    all_generator = torch.utils.data.DataLoader(full_dataset, shuffle=False, batch_size=eval_batchsize)
    
multi_model_record=None
global_models=None
global_summary_model=None

os.makedirs(save_folder, exist_ok=True)
save_plot=save_folder+f"/summary_ep_{n_Epochs}.png"
print("Will save models in:\n\t", save_folder)

Found normalization factors across all ligands 2023-11-17 00:15:42.647392
Will save models in:
	 models/Perceptron
